In [1]:
import os

os.environ['PROJ_LIB'] = 'C:/Users/tseng/anaconda3/pkgs/proj-7.2.0-h3e70539_0/Library/share'
# There's an error about PROJ_LIB when running the below lib calls so I added this part

In [2]:
# standard libs
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json

# plotting libs
import seaborn as sns

# geospatial libs
from mpl_toolkits.basemap import Basemap
from shapely.geometry import Polygon
#import geopandas as gpd
import folium
import plotly.graph_objects as go
import plotly_express as px

# set in line plotly 
from plotly.offline import init_notebook_mode;
init_notebook_mode(connected=True)

print(os.getcwd())
pd.set_option('mode.chained_assignment',None)

C:\Users\tseng\OneDrive - KU Leuven\MDA\Python


In [3]:
# import cities response df
cities_df = pd.read_csv(r'2020_Full_Cities_Dataset.csv')

In [4]:
Cities_disclosing_df = pd.read_csv(r'2020_Cities_Disclosing_to_CDP.csv')
Cities_disclosing_df.drop(['Year Reported to CDP','Organization', 'Reporting Authority', 'Access', 'First Time Discloser', 'Population Year','Last update'], axis = 1, inplace = True)

In [5]:
MPI = pd.read_csv(r'Water poverty index.csv') 
MPI.drop(['the_geom','FID','OBJECTID','ISO','UNESCOMEMB'], axis = 1, inplace = True)
MPI.columns = ['Country', 'WPI']
MPI['Country'] =MPI['Country'].replace({"Cote d'Ivoire":"Côte d'Ivoire"})
MPI['Country'] =MPI['Country'].replace({"Palestine":"State of Palestine"})
MPI.loc[MPI.shape[0], 'Country'] = 'Taiwan, Greater China'
MPI.loc[MPI.shape[0]-1, 'WPI'] = 56.6
MPI.loc[MPI.shape[0], 'Country'] = 'Gibraltar'
MPI.loc[MPI.shape[0]-1, 'WPI'] = -999.0
MPI.loc[MPI.shape[0], 'Country'] = 'China, Hong Kong Special Administrative Region'
MPI.loc[MPI.shape[0]-1, 'WPI'] = 56.6

In [6]:
merge_df = pd.merge(Cities_disclosing_df, MPI)
merge_df['Population'][548]=1383432134.0/1000
merge_df['Population'][396]=399724000.0/100
merge_df.sort_values(by = 'Population', inplace = True)

In [7]:
def plot_risk_action(risk = 'Inadequate or ageing water supply infrastructure', action = 'Investment in existing water supply infrastructure'):
    negate_action = "No " + str(action)
    text_action = str(action)
    global merge_df
    Q143 = cities_df[cities_df['Question Number'] == '14.3']
    Q143w = Q143[Q143['Response Answer']==risk]

    Q143_actions = pd.DataFrame()

    for index in Q143w.index:
        Row_Number = Q143['Row Number'][index]
        Account_Number = Q143['Account Number'][index]
        df1 = Q143[Q143['Account Number']==Account_Number]
        df1 = df1[df1['Row Number']==Row_Number]
        Q143_actions = pd.concat([Q143_actions,df1])

    Q143_actions = Q143_actions[Q143_actions['Response Answer']== action]

    Q143_status_action = pd.DataFrame()
    for index in Q143_actions.index:
        Row_Number = Q143['Row Number'][index]
        Account_Number = Q143['Account Number'][index]
        df2 = Q143[Q143['Account Number']==Account_Number]
        df2 = df2[df2['Row Number']==Row_Number]
        Q143_status_action = pd.concat([Q143_status_action,df2])

    Q143_status_action1 = Q143_status_action[Q143_status_action['Response Answer']=='Scoping']
    Q143_status_action2 = Q143_status_action[Q143_status_action['Response Answer']=='Pre-feasibility study/impact assessment']
    Q143_status_action3 = Q143_status_action[Q143_status_action['Response Answer']=='Pre-implementation']
    Q143_status_action4 = Q143_status_action[Q143_status_action['Response Answer']=='Implementation']
    Q143_status_action5 = Q143_status_action[Q143_status_action['Response Answer']=='Implementation complete but not in operation']
    Q143_status_action6 = Q143_status_action[Q143_status_action['Response Answer']=='Operation']
    Q143_status_action7 = Q143_status_action[Q143_status_action['Response Answer']=='Monitoring and reporting']

    merge_df['Risks'] = 'Not identified as a risk'
    for index in merge_df.index:
        for j in Q143w.index:
            Account_Number = Q143w['Account Number'][j]
            if merge_df['Account Number'][index]==Account_Number:
                for k in Q143_actions.index :
                    action = Q143_actions['Account Number'][k]
                    if merge_df['Account Number'][index]==action:
                        merge_df['Risks'][index]= negate_action
                        for h in Q143_status_action1.index:
                            status =Q143_status_action1['Account Number'][h]
                            if merge_df['Account Number'][index]==status: 
                                merge_df['Risks'][index]='Scoping'
                        for h in Q143_status_action2.index:
                            status =Q143_status_action2['Account Number'][h]
                            if merge_df['Account Number'][index]==status: 
                                merge_df['Risks'][index]='Pre-feasibility study/impact assessment'
                        for h in Q143_status_action3.index:
                            status =Q143_status_action3['Account Number'][h]
                            if merge_df['Account Number'][index]==status:
                                merge_df['Risks'][index]='Pre-implementation'
                        for h in Q143_status_action4.index:
                            status =Q143_status_action4['Account Number'][h]
                            if merge_df['Account Number'][index]==status:
                                merge_df['Risks'][index]='Implementation'
                        for h in Q143_status_action5.index:
                            status =Q143_status_action5['Account Number'][h]
                            if merge_df['Account Number'][index]==status:
                                merge_df['Risks'][index]='Implementation complete but not in operation'
                        for h in Q143_status_action6.index:
                            status =Q143_status_action6['Account Number'][h]
                            if merge_df['Account Number'][index]==status:
                                merge_df['Risks'][index]='Operation'
                        for h in Q143_status_action7.index:
                            status =Q143_status_action7['Account Number'][h]
                            if merge_df['Account Number'][index]==status: 
                                merge_df['Risks'][index]='Monitoring and reporting'

    merge_df['Population'].fillna(0, inplace = True) 
    merge_df = merge_df[merge_df['Population']!=0]

    merge_df['City Location'].fillna(0, inplace = True) 
    merge_df = merge_df[merge_df['City Location']!=0]
    merge_df['longitude'] = merge_df['City Location']
    merge_df['latitude'] = merge_df['City Location']
    for i in merge_df.index:
        merge_df['longitude'][i] = merge_df['City Location'][i].split()[1].replace('(','')
        merge_df['latitude'][i] = merge_df['City Location'][i].split()[2].replace(')','')

    Fig_Q14_3 = px.scatter_geo(merge_df,lat = 'latitude',lon= 'longitude',size = 'Population',width=1200,height = 1000, color='Risks', hover_name="City",projection="natural earth",color_discrete_map={
                    negate_action: "#C4181C",
                    'Scoping': "#EC4B2E",
                    'Pre-feasibility study/impact assessment': "#FA9346",
                    'Pre-implementation': "#FBD374",
                    'Implementation': "#CBEC6E",
                    'Implementation complete but not in operation': "#A1D868",
                    'Operation': "#1A7E37",
                    'Monitoring and reporting': "#5992BE",
                    'Not identified as a risk': "#2E559B"},
                    category_orders={'Risks':(
                    negate_action,
                    'Scoping',
                    'Pre-feasibility study/impact assessment',
                    'Pre-implementation',
                    'Implementation',
                    'Implementation complete but not in operation',
                    'Operation',
                    'Monitoring and reporting',
                    'Not identified as a risk')
                    },  
                    hover_data = {'longitude':False,'latitude':False,'Country':True,'Population':True,'CDP Region':True,'Risks':True, 'WPI': True},
                    labels = {'Risks':'Status of action'})
    Fig_Q14_3.update_layout(
        title={
            'text': "Risk to urban water security: " + str(risk) ,
            'y':0.85,
            'x':0.35,
            'xanchor': 'center',
            'yanchor': 'top'},
        legend_title={
            'text': "Action: " + str(text_action),
            'side': "top"},
        legend = {
            'yanchor':"top",
            'y':0.9,
            'xanchor':"left",
            'x':1},
        font=dict(
            family="Arial",
            size=12,
            color= "black"),
        autosize=False,
        width=800,
        height=400,
    )
    Fig_Q14_3 .show()

In [8]:
#plot_risk_action(risk = 'Rationing of municipal water supply', action = 'Conservation awareness and education')
#plot_risk_action(risk = 'Rationing of municipal water supply', action = 'Conservation incentives Water use restrictions')
#plot_risk_action(risk = 'Rationing of municipal water supply', action = 'Efficiency regulations or standards Water metering')
plot_risk_action(risk = 'Rationing of municipal water supply', action = 'Municipal water efficiency retrofits')
plot_risk_action(risk = 'Rationing of municipal water supply', action = 'Diversifying water supply (including new sources)')
plot_risk_action(risk = 'Rationing of municipal water supply', action = 'Investment in existing water supply infrastructure')
#plot_risk_action(risk = 'Rationing of municipal water supply', action = 'Use of non-potable water outside')
#plot_risk_action(risk = 'Rationing of municipal water supply', action = 'Use of non-potable water indoors (within building) Watershed preservation')
plot_risk_action(risk = 'Rationing of municipal water supply', action = 'Stormwater management (natural or man-made infrastructure)')
#plot_risk_action(risk = 'Rationing of municipal water supply', action = 'Nature-based solution Water recycling/reclamation')
plot_risk_action(risk = 'Severe weather events', action = 'Conservation awareness and education')
#plot_risk_action(risk = 'Severe weather events', action = 'Conservation incentives Water use restrictions')
#plot_risk_action(risk = 'Severe weather events', action = 'Efficiency regulations or standards Water metering')
#plot_risk_action(risk = 'Severe weather events', action = 'Municipal water efficiency retrofits')
plot_risk_action(risk = 'Severe weather events', action = 'Diversifying water supply (including new sources)')
plot_risk_action(risk = 'Severe weather events', action = 'Investment in existing water supply infrastructure')
#plot_risk_action(risk = 'Severe weather events', action = 'Use of non-potable water outside')
#plot_risk_action(risk = 'Severe weather events', action = 'Use of non-potable water indoors (within building) Watershed preservation')
plot_risk_action(risk = 'Severe weather events', action = 'Stormwater management (natural or man-made infrastructure)')
#plot_risk_action(risk = 'Severe weather events', action = 'Nature-based solution Water recycling/reclamation')
plot_risk_action(risk = 'Ecosystem vulnerability', action = 'Conservation awareness and education')
#plot_risk_action(risk = 'Ecosystem vulnerability', action = 'Conservation incentives Water use restrictions')
#plot_risk_action(risk = 'Ecosystem vulnerability', action = 'Efficiency regulations or standards Water metering')
plot_risk_action(risk = 'Ecosystem vulnerability', action = 'Municipal water efficiency retrofits')
#plot_risk_action(risk = 'Ecosystem vulnerability', action = 'Diversifying water supply (including new sources)')
#plot_risk_action(risk = 'Ecosystem vulnerability', action = 'Investment in existing water supply infrastructure')
#plot_risk_action(risk = 'Ecosystem vulnerability', action = 'Use of non-potable water outside')
#plot_risk_action(risk = 'Ecosystem vulnerability', action = 'Use of non-potable water indoors (within building) Watershed preservation')
#plot_risk_action(risk = 'Ecosystem vulnerability', action = 'Stormwater management (natural or man-made infrastructure)')
#plot_risk_action(risk = 'Ecosystem vulnerability', action = 'Nature-based solution Water recycling/reclamation')
plot_risk_action(risk = 'Change in land-use', action = 'Conservation awareness and education')
#plot_risk_action(risk = 'Change in land-use', action = 'Conservation incentives Water use restrictions')
#plot_risk_action(risk = 'Change in land-use', action = 'Efficiency regulations or standards Water metering')
#plot_risk_action(risk = 'Change in land-use', action = 'Municipal water efficiency retrofits')
plot_risk_action(risk = 'Change in land-use', action = 'Diversifying water supply (including new sources)')
#plot_risk_action(risk = 'Change in land-use', action = 'Investment in existing water supply infrastructure')
#plot_risk_action(risk = 'Change in land-use', action = 'Use of non-potable water outside')
#plot_risk_action(risk = 'Change in land-use', action = 'Use of non-potable water indoors (within building) Watershed preservation')
plot_risk_action(risk = 'Change in land-use', action = 'Stormwater management (natural or man-made infrastructure)')
#plot_risk_action(risk = 'Change in land-use', action = 'Nature-based solution Water recycling/reclamation')
plot_risk_action(risk = 'Pollution incidents', action = 'Conservation awareness and education')
#plot_risk_action(risk = 'Pollution incidents', action = 'Conservation incentives Water use restrictions')
#plot_risk_action(risk = 'Pollution incidents', action = 'Efficiency regulations or standards Water metering')
#plot_risk_action(risk = 'Pollution incidents', action = 'Municipal water efficiency retrofits')
#plot_risk_action(risk = 'Pollution incidents', action = 'Diversifying water supply (including new sources)')
plot_risk_action(risk = 'Pollution incidents', action = 'Investment in existing water supply infrastructure')
#plot_risk_action(risk = 'Pollution incidents', action = 'Use of non-potable water outside')
#plot_risk_action(risk = 'Pollution incidents', action = 'Use of non-potable water indoors (within building) Watershed preservation')
plot_risk_action(risk = 'Pollution incidents', action = 'Stormwater management (natural or man-made infrastructure)')
#plot_risk_action(risk = 'Pollution incidents', action = 'Nature-based solution Water recycling/reclamation')
#plot_risk_action(risk = 'Increased levels of plastic in freshwater bodies Water infestation/disease', action = 'Conservation awareness and education')
#plot_risk_action(risk = 'Increased levels of plastic in freshwater bodies Water infestation/disease', action = 'Conservation incentives Water use restrictions')
#plot_risk_action(risk = 'Increased levels of plastic in freshwater bodies Water infestation/disease', action = 'Efficiency regulations or standards Water metering')
#plot_risk_action(risk = 'Increased levels of plastic in freshwater bodies Water infestation/disease', action = 'Municipal water efficiency retrofits')
#plot_risk_action(risk = 'Increased levels of plastic in freshwater bodies Water infestation/disease', action = 'Diversifying water supply (including new sources)')
#plot_risk_action(risk = 'Increased levels of plastic in freshwater bodies Water infestation/disease', action = 'Investment in existing water supply infrastructure')
#plot_risk_action(risk = 'Increased levels of plastic in freshwater bodies Water infestation/disease', action = 'Use of non-potable water outside')
#plot_risk_action(risk = 'Increased levels of plastic in freshwater bodies Water infestation/disease', action = 'Use of non-potable water indoors (within building) Watershed preservation')
#plot_risk_action(risk = 'Increased levels of plastic in freshwater bodies Water infestation/disease', action = 'Stormwater management (natural or man-made infrastructure)')
#plot_risk_action(risk = 'Increased levels of plastic in freshwater bodies Water infestation/disease', action = 'Nature-based solution Water recycling/reclamation')
#plot_risk_action(risk = 'Unauthorized/unregistered water connections', action = 'Conservation awareness and education')
#plot_risk_action(risk = 'Unauthorized/unregistered water connections', action = 'Conservation incentives Water use restrictions')
#plot_risk_action(risk = 'Unauthorized/unregistered water connections', action = 'Efficiency regulations or standards Water metering')
#plot_risk_action(risk = 'Unauthorized/unregistered water connections', action = 'Municipal water efficiency retrofits')
#plot_risk_action(risk = 'Unauthorized/unregistered water connections', action = 'Diversifying water supply (including new sources)')
#plot_risk_action(risk = 'Unauthorized/unregistered water connections', action = 'Investment in existing water supply infrastructure')
#plot_risk_action(risk = 'Unauthorized/unregistered water connections', action = 'Use of non-potable water outside')
#plot_risk_action(risk = 'Unauthorized/unregistered water connections', action = 'Use of non-potable water indoors (within building) Watershed preservation')
#plot_risk_action(risk = 'Unauthorized/unregistered water connections', action = 'Stormwater management (natural or man-made infrastructure)')
#plot_risk_action(risk = 'Unauthorized/unregistered water connections', action = 'Nature-based solution Water recycling/reclamation')